In [1]:
import argparse
import torch
import dgl
from dgl.data import CoraGraphDataset, CiteseerGraphDataset, PubmedGraphDataset
from model import LabelPropagation

Using backend: pytorch


In [2]:
def main():
    # check cuda
    device = f'cuda:{args.gpu}' if torch.cuda.is_available() and args.gpu >= 0 else 'cpu'

    # load data
    if args.dataset == 'Cora':
        dataset = CoraGraphDataset()
    elif args.dataset == 'Citeseer':
        dataset = CiteseerGraphDataset()
    elif args.dataset == 'Pubmed':
        dataset = PubmedGraphDataset()
    else:
        raise ValueError('Dataset {} is invalid.'.format(args.dataset))
    
    g = dataset[0]
    # A + I 操作 
    g = dgl.add_self_loop(g)

    labels = g.ndata.pop('label').to(device).long()

    # load masks for train / test, valid is not used.
    train_mask = g.ndata.pop('train_mask')
    test_mask = g.ndata.pop('test_mask')
    
    # 得到非0的索引(index)
    train_idx = torch.nonzero(train_mask, as_tuple=False).squeeze().to(device)
    test_idx = torch.nonzero(test_mask, as_tuple=False).squeeze().to(device)

    g = g.to(device)
    
    # label propagation
    # 定义模型 
    lp = LabelPropagation(args.num_layers, args.alpha)
    # 调用模型
    logits = lp(g, labels, mask=train_idx)

    # 计算准确率
    test_acc = torch.sum(logits[test_idx].argmax(dim=1) == labels[test_idx]).item() / len(test_idx)
    print("Test Acc {:.4f}".format(test_acc))

In [3]:
#if __name__ == '__main__':
"""
Label Propagation Hyperparameters
"""
parser = argparse.ArgumentParser(description='LP')
parser.add_argument('--gpu', type=int, default=0)
parser.add_argument('--dataset', type=str, default='Cora')
parser.add_argument('--num-layers', type=int, default=10)
parser.add_argument('--alpha', type=float, default=0.5)

#args = parser.parse_args()
args = parser.parse_args(args=[])
print(args)

main()

Namespace(alpha=0.5, dataset='Cora', gpu=0, num_layers=10)
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Test Acc 0.6920
